In [1]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue
from math import ceil

import numpy as np

In [18]:
PROBLEM_SIZE = 30
NUM_SETS = 50
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken', 'not_taken'])

In [6]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))


In [7]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Problem not solvable"

In [10]:
def h(state):
  # current set (with all covered tiles)
  already_covered = reduce(
    np.logical_or,
    [SETS[i] for i in state.taken],
    np.array([False for _ in range(PROBLEM_SIZE)]),
  )
  
  if np.all(already_covered):
    return 0
  
  missing_elements = PROBLEM_SIZE - sum(already_covered)
  
  largest_size_set = 0
  for i in state.not_taken:
    uncovered_elements = np.logical_and(SETS[i], np.logical_not(already_covered))          # uncovered elements by i-th set (they are True now)
    size = sum(uncovered_elements)                      # counting all True values (that were False before)
    largest_size_set = max(largest_size_set, size)      # selecting the maximum number of False in sets
    
  if(largest_size_set != 0):
    result = ceil(missing_elements / largest_size_set)
  else:
    result = missing_elements+1

  return result
  
def g(state):
  return len(state.taken)

def f(state):
  return g(state) + h(state)

In [19]:
frontier = PriorityQueue()
#state = State(set(), set(range(NUM_SETS)))
actions = filter(lambda x: sum(SETS[x]) != 0, range(NUM_SETS))
frontier.put( (0, State(set(), set(actions))) )

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put( (f(new_state), new_state) )
    _, current_state = frontier.get()

#[print(SETS[i]) for i in current_state.taken]
print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

Solved in 56 steps (3 tiles)
